**What it is**:  
  - This end-to-end script defines, builds, and loads structured portfolio metadata (Portfolio General Information) for a REIT-focused investment system. It includes portfolio names, styles, inception details, and currency configuration.

**What it does**:  
  - **Defines portfolios** statically in a Python dictionary with fields like name, investment style, and product code  
  - **Builds the PGI DataFrame (`pgi_df`)**, assigning REIT-specific metadata, open dates, and currency information  
  - **Formats and cleans** the date columns to ensure compatibility with Snowflake  
  - **Connects to Snowflake** and creates a temporary staging table (`STG_PGI`)  
  - **Uploads the data** using `write_pandas()`  
  - **Merges into the main `PortfolioGeneralInformation` table**, ensuring existing portfolios are updated and new ones are inserted

**Why it's important**:  
  - Provides a single source of truth for portfolio metadata  
  - Supports consistent data modeling across the platform  
  - Enables accurate linkage with benchmarks, performance tracking, and downstream analytics  
  - Designed to be extendable for new portfolios or evolving business requirements


# **Portfolio General Information Table**

## Populating PGI Table and mapping portfolio codes {FINAL}

In [ ]:
import requests, pandas as pd, time
from bs4 import BeautifulSoup
from typing import List

# Custom user-agent for web requests
UA = {"User-Agent": "Mozilla/5.0 (MSBA Capstone Bot/1.0)"}

def load_sec_master() -> pd.DataFrame:
    """
    Loads ticker and CIK mapping from the SEC's JSON feed.

    Returns:
        DataFrame with standardized 'Ticker' and zero-padded 'CIK'.
    """
    url = "https://www.sec.gov/files/company_tickers_exchange.json"
    data = requests.get(url, headers=UA, timeout=30).json()
    fields = data["fields"]
    df = pd.DataFrame(data["data"], columns=fields)
    df["Ticker"] = df["ticker"].str.upper()
    df["CIK"] = df["cik"].astype(str).str.zfill(10)
    return df[["Ticker", "CIK"]]

def load_reitnotes() -> pd.DataFrame:
    """
    Scrapes REIT types from REITNotes.com.

    Returns:
        DataFrame with 'Ticker' and simplified 'NotesType' (Equity or Mortgage).
    """
    url = "https://www.reitnotes.com/reit-list/"
    html = requests.get(url, headers=UA, timeout=30).text
    df = pd.read_html(html)[0][["Symbol", "Type"]].rename(columns={"Symbol": "Ticker", "Type": "NotesType"})
    df["Ticker"] = df["Ticker"].str.upper()
    df["NotesType"] = df["NotesType"].str.replace(" REIT", "", regex=False)
    df["NotesType"] = df["NotesType"].map({"Equity": "Equity", "Mortgage": "Mortgage"})
    return df

def load_nareit(max_pages=25) -> pd.DataFrame:
    """
    Scrapes REIT investment styles from Nareit.org directory pages.

    Args:
        max_pages: Maximum number of paginated results to fetch.

    Returns:
        DataFrame with 'Ticker' and 'NareitStyle' (Equity or Mortgage).
    """
    rows = []
    for page in range(max_pages):
        url = f"https://www.reit.com/investing/reit-directory?page={page}"
        html = requests.get(url, headers=UA, timeout=30).text
        soup = BeautifulSoup(html, "lxml")
        table = soup.find("table")
        if not table or not table.tbody:
            break  # No more data
        for tr in table.tbody.find_all("tr"):
            tds = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
            if len(tds) < 3:
                continue
            ticker_raw = tds[1].split("|")[0].strip()
            style = "Mortgage" if tds[-1].lower() == "mortgage" else "Equity"
            rows.append({"Ticker": ticker_raw.upper(), "NareitStyle": style})
        time.sleep(0.1)  # Be kind to the server
    return pd.DataFrame(rows).drop_duplicates()

def resolve_style(row) -> str | None:
    """
    Resolves the final REIT investment style by reconciling values from REITNotes and Nareit.

    Args:
        row: A single DataFrame row containing NotesType and NareitStyle.

    Returns:
        Investment style: Equity, Mortgage, Hybrid, or None.
    """
    a, b = row["NotesType"], row["NareitStyle"]
    if pd.isna(a) and pd.isna(b):
        return None
    if a == b:
        return a
    if pd.isna(a):
        return b
    if pd.isna(b):
        return a
    return "Hybrid"

def classify_styles(tickers: List[str]) -> pd.DataFrame:
    """
    Classifies investment styles for a list of REIT tickers using external data sources.

    Args:
        tickers: List of ticker symbols.

    Returns:
        DataFrame with 'Ticker' and resolved 'InvestmentStyle'.
    """
    sec = load_sec_master()
    notes = load_reitnotes()
    nareit = load_nareit()

    base = pd.DataFrame({"Ticker": [t.upper() for t in tickers]})
    merged = (
        base
        .merge(sec, on="Ticker", how="left")
        .merge(notes, on="Ticker", how="left")
        .merge(nareit, on="Ticker", how="left")
    )
    merged["InvestmentStyle"] = merged.apply(resolve_style, axis=1)
    return merged[["Ticker", "InvestmentStyle"]]


# --- Build PortfolioGeneralInformation Table ---

# Define static portfolios with associated metadata
portfolios = {
    "REITPORTFOLIO001": {
        "NAME": "Core U.S. Equity REITs",
        "PRODUCTCODE": "REITProd001",
        "INVESTMENTSTYLE": "Equity"
    },
    "REITPORTFOLIO002": {
        "NAME": "Real Estate Services & Platforms",
        "PRODUCTCODE": "REITProd001",
        "INVESTMENTSTYLE": "Equity"
    },
    "REITPORTFOLIO003": {
        "NAME": "Global Real Estate Diversified",
        "PRODUCTCODE": "REITProd002",
        "INVESTMENTSTYLE": "Equity"
    },
    "REITPORTFOLIO004": {
        "NAME": "Mortgage & Hybrid REITs",
        "PRODUCTCODE": "REITProd002",
        "INVESTMENTSTYLE": "Hybrid"
    },
    "REITPORTFOLIO005": {
        "NAME": "Industrial, Retail & Infrastructure",
        "PRODUCTCODE": "REITProd001",
        "INVESTMENTSTYLE": "Equity"
    }
}

# Construct each row of PortfolioGeneralInformation
rows = []
for code, meta in portfolios.items():
    rows.append({
        "PORTFOLIOCODE": code,
        "NAME": meta["NAME"],
        "INVESTMENTSTYLE": meta["INVESTMENTSTYLE"],
        "PORTFOLIOCATEGORY": "REIT",
        "OPENDATE": pd.to_datetime("2020-01-01"),
        "PERFORMANCEINCEPTIONDATE": pd.to_datetime("2020-01-01"),
        "TERMINATIONDATE": pd.NaT,
        "BASECURRENCYCODE": "USD",
        "BASECURRENCYNAME": "US Dollar",
        "ISBEGINOFDAYPERFORMANCE": True,
        "PRODUCTCODE": meta["PRODUCTCODE"]
    })

# Convert to final PGI DataFrame
pgi_df = pd.DataFrame(rows)
print(pgi_df)


      PORTFOLIOCODE                                 NAME INVESTMENTSTYLE  \
0  REITPORTFOLIO001               Core U.S. Equity REITs          Equity   
1  REITPORTFOLIO002     Real Estate Services & Platforms          Equity   
2  REITPORTFOLIO003       Global Real Estate Diversified          Equity   
3  REITPORTFOLIO004              Mortgage & Hybrid REITs          Hybrid   
4  REITPORTFOLIO005  Industrial, Retail & Infrastructure          Equity   

  PORTFOLIOCATEGORY   OPENDATE PERFORMANCEINCEPTIONDATE TERMINATIONDATE  \
0              REIT 2020-01-01               2020-01-01             NaT   
1              REIT 2020-01-01               2020-01-01             NaT   
2              REIT 2020-01-01               2020-01-01             NaT   
3              REIT 2020-01-01               2020-01-01             NaT   
4              REIT 2020-01-01               2020-01-01             NaT   

  BASECURRENCYCODE BASECURRENCYNAME  ISBEGINOFDAYPERFORMANCE  PRODUCTCODE  
0              U

In [ ]:
pgi_df.to_csv('pgi_df.csv', index=False)

# **Functional Code + Sample Validation, Error Handling, Incomplete Data + Edge Case Handling**

This code builds on the above functional code, but is broken out into separate scripts which can be put into .py files and run with Assette's larger Python scripts. Also, it has sample code for the above categories.

- Steps 1 & 2 are breakdowns of the functional code from section "Combining approaches..." > "Populating PGI Table and mapping portfolio codes {FINAL}"

- Steps 3 & 4 are the sample code for validation, errors, incompletions, and edge cases

Note that these code blocks should be saved to .py files with their subheader name and structured into a folder with a module layout, within an ETL pipeline.

## 1) classify_styles.py

In [ ]:
"""
Purpose:

 - Grabs the master list of all public U.S. companies from the SEC, including:
    - Ticker (e.g., "O", "PLD")
    - CIK (central index key, e.g., "0000726728")

How it works:

- Pulls from https://www.sec.gov/files/company_tickers_exchange.json
- Parses the JSON response into a pandas.DataFrame
- Standardizes the formatting (uppercase tickers, padded CIKs)

Why it's important:
- Ensures pipeline works for any ticker — not just REITs
- CIKs are useful later to query full SEC filings or facts

"""

# classify_styles.py
import requests, pandas as pd, time
from bs4 import BeautifulSoup
from typing import List

UA = {"User-Agent": "Mozilla/5.0 (MSBA Capstone Bot/1.0)"}

def load_sec_master():
    url = "https://www.sec.gov/files/company_tickers_exchange.json"
    data = requests.get(url, headers=UA, timeout=30).json()
    fields = data["fields"]
    df = pd.DataFrame(data["data"], columns=fields)
    df["Ticker"] = df["ticker"].str.upper()
    df["CIK"] = df["cik"].astype(str).str.zfill(10)
    return df[["Ticker", "CIK"]]

def load_reitnotes():
    url = "https://www.reitnotes.com/reit-list/"
    html = requests.get(url, headers=UA, timeout=30).text
    df = pd.read_html(html)[0][["Symbol", "Type"]]
    df.columns = ["Ticker", "NotesType"]
    df["Ticker"] = df["Ticker"].str.upper()
    df["NotesType"] = df["NotesType"].str.replace(" REIT", "", regex=False)
    return df[df["NotesType"].isin(["Equity", "Mortgage"])]

def load_nareit(max_pages=25):
    rows = []
    for page in range(max_pages):
        url = f"https://www.reit.com/investing/reit-directory?page={page}"
        html = requests.get(url, headers=UA, timeout=30).text
        soup = BeautifulSoup(html, "lxml")
        table = soup.find("table")
        if not table or not table.tbody:
            break
        for tr in table.tbody.find_all("tr"):
            tds = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
            if len(tds) < 3:
                continue
            ticker = tds[1].split("|")[0].strip().upper()
            style = "Mortgage" if tds[-1].lower() == "mortgage" else "Equity"
            rows.append({"Ticker": ticker, "NareitStyle": style})
        time.sleep(0.1)
    return pd.DataFrame(rows).drop_duplicates()

def resolve_style(row):
    a, b = row["NotesType"], row["NareitStyle"]
    if pd.isna(a) and pd.isna(b):
        return None
    if a == b:
        return a
    if pd.isna(a):
        return b
    if pd.isna(b):
        return a
    return "Hybrid"

def classify_styles(tickers: List[str]) -> pd.DataFrame:
    sec = load_sec_master()
    notes = load_reitnotes()
    nareit = load_nareit()

    base = pd.DataFrame({"Ticker": [t.upper() for t in tickers]})
    merged = (
        base.merge(sec, on="Ticker", how="left")
            .merge(notes, on="Ticker", how="left")
            .merge(nareit, on="Ticker", how="left")
    )
    merged["InvestmentStyle"] = merged.apply(resolve_style, axis=1)
    return merged[["Ticker", "InvestmentStyle"]]


## 2) generate_pgi_df.py

In [ ]:
"""
Purpose: Generates synthetic PortfolioGeneralInformation (PGI) data and organizing them under synthetic portfolios.

How it works:
- Creates 5 Portfolios based on common ticker traits
- Maps the 5 portfolios to each of 2 synthetic products (REITProd001 and REITProd002)
- Constructs a PGI dataframe with: • Portfolio metadata (e.g. OPENDATE, CURRENCY, CATEGORY, etc.) • InvestmentStyle classification for each portfolio

Why it’s important:
- Enables Assette to simulate realistic REIT portfolio data.
- Fully populates required columns for the PortfolioGeneralInformation table in Snowflake.
- Modularized for Colab-based prototyping and Snowflake integration.

NOTE: This code block will not run in Colab because the local environment will not recognize "classify_styles", etc. as traditional packages. It will work in a Python pipeline where:
  - The .py files are stored together in a defined directory
  - The Python environment is executed as a package or script
  - All modules (reconcile_styles.py, validate.py, main.py, etc.) are treated as first-class imports

"""

# generate_pgi.py
import pandas as pd
from classify_styles import classify_styles

# Define static portfolios
# Define static portfolios
portfolios = {
    "REITPORTFOLIO001": {
        "NAME": "Core U.S. Equity REITs",
        "PRODUCTCODE": "REITProd001",
        "INVESTMENTSTYLE": "Equity"
    },
    "REITPORTFOLIO002": {
        "NAME": "Real Estate Services & Platforms",
        "PRODUCTCODE": "REITProd001",
        "INVESTMENTSTYLE": "Equity"
    },
    "REITPORTFOLIO003": {
        "NAME": "Global Real Estate Diversified",
        "PRODUCTCODE": "REITProd002",
        "INVESTMENTSTYLE": "Equity"
    },
    "REITPORTFOLIO004": {
        "NAME": "Mortgage & Hybrid REITs",
        "PRODUCTCODE": "REITProd002",
        "INVESTMENTSTYLE": "Hybrid"
    },
    "REITPORTFOLIO005": {
        "NAME": "Industrial, Retail & Infrastructure",
        "PRODUCTCODE": "REITProd001",
        "INVESTMENTSTYLE": "Equity"
    }
}


# Build PGI table statically
rows = []
for code, meta in portfolios.items():
    rows.append({
        "PORTFOLIOCODE": code,
        "NAME": meta["NAME"],
        "INVESTMENTSTYLE": meta["INVESTMENTSTYLE"],
        "PORTFOLIOCATEGORY": "REIT",
        "OPENDATE": pd.to_datetime("2020-01-01"),
        "PERFORMANCEINCEPTIONDATE": pd.to_datetime("2020-01-01"),
        "TERMINATIONDATE": pd.NaT,
        "BASECURRENCYCODE": "USD",
        "BASECURRENCYNAME": "US Dollar",
        "ISBEGINOFDAYPERFORMANCE": True,
        "PRODUCTCODE": meta["PRODUCTCODE"]
    })

# Final DataFrame
pgi_df = pd.DataFrame(rows)


## 3) validate.py

In [ ]:
"""
Purpose:
- Adds data quality checks and optional retry logic for network calls.

What it does:

- Validates that each InvestmentStyle value is part of an approved list (including REIT and non-REIT asset classes).
- Logs a warning if any styles are missing (None) and suggests further investigation
- Logs an error and raises an exception if any styles are invalid or unrecognized
- Raises a critical alert if more than 30% of tickers have missing InvestmentStyle values
- Writes all validation results to a structured log file (investmentstyle_validation.log) with timestamps, function names, and line numbers
- Handles portfolio-level validation (not just tickers)
- Refers to the correct column name INVESTMENTSTYLE (used in the PGI table)
- Expands the log to include the PORTFOLIOCODE for any invalid entries
- Provides clear, structured logging and critical failure detection

Why it's important:

- Keeps data clean and predictable
- Helps to make the pipeline more fault-tolerant during web scraping or API flakiness
- Future-looking & inclusive to integrate other asset classes that Assette may encounter in their environment


"""

# validate.py
import pandas as pd
import logging

# Valid styles for REITs and other asset classes
VALID_STYLES = {
    "Equity REIT", "Mortgage REIT", "Hybrid REIT",
    "Private Equity", "Hedge Fund", "Mutual Fund",
    "Alternative Asset", "Multi-Asset",
    "Other", None
}

# Set up logging
logging.basicConfig(
    filename="style_classification.log",
    filemode="a",
    format="%(asctime)s | %(levelname)s | %(funcName)s | line %(lineno)d | %(message)s",
    level=logging.INFO
)

def assert_styles(df: pd.DataFrame) -> None:
    """
    Validates the INVESTMENTSTYLE column in the PGI table.
    Logs missing or invalid values and raises an error if thresholds are exceeded.
    """

    total = len(df)
    missing = df["INVESTMENTSTYLE"].isna().sum()
    invalid = df[~df["INVESTMENTSTYLE"].isin(VALID_STYLES)]

    # Logging summary
    logging.info(f"🧪 Validating {total} portfolios in INVESTMENTSTYLE...")

    if missing > 0:
        logging.warning(f"{missing} portfolios have missing INVESTMENTSTYLE values.")

    if not invalid.empty:
        logging.error(f"{len(invalid)} portfolios have invalid INVESTMENTSTYLE values:")
        logging.error(f"\n{invalid[['PORTFOLIOCODE', 'INVESTMENTSTYLE']].to_string(index=False)}")

    # Raise error if more than 30% of styles are missing
    if missing / total > 0.3:
        raise ValueError(f"Validation failed: Too many missing styles ({missing} / {total})")

    # Raise error on invalid styles
    if not invalid.empty:
        raise ValueError("Validation failed: Invalid INVESTMENTSTYLE values found.")

    if missing == 0 and invalid.empty:
        logging.info("INVESTMENTSTYLE validation passed with no issues.")
    elif missing > 0:
        logging.info("ℹSuggest reviewing undefined styles — may need external classification.")


## 4) main.py

In [ ]:
# main.py
"""
Main entry point for generating and uploading PGI data for REIT portfolios.

Purpose:
Entry point that runs the full pipeline and optionally loads the result into Snowflake.

What it does:
Calls all extractors and reconciliation functions
Validates the final DataFrame
Joins with a mock PortfolioGeneralInformation table
Prints the result (and optionally writes it to Snowflake using write_pandas())

Why it's important:

Ties all steps together
Can be scheduled or called from a larger system like Assette

NOTE: This code block will not run in Colab because the local environment will not recognize "classify_styles", etc. as traditional packages.
It will work in a Python pipeline where:
- The .py files are stored together in a defined directory
- The Python environment is executed as a package or script
All modules (reconcile_styles.py, validate.py, main.py, etc.) are treated as first-class imports

"""

import pandas as pd
from classify_style import classify_styles
from generate_pgi_df import generate_pgi_df
from validate import assert_styles
from snowflake_loader import load_to_snowflake

# Step 1: Classify investment styles
style_df = classify_styles(reit_tickers)

# Step 2: Generate static PGI records
pgi_df = generate_pgi_df()

# Step 3: Validate style assignments
assert_styles(style_df)

# Step 4: Upload to Snowflake
load_to_snowflake(pgi_df)



# **Connecting to Snowflake & Loading in Data**

## Establishing Connection

You must download the env.txt file which is located in my GDrive folder ("Shiv-Code") and upload using the "Choose Files" widget after running the below cell.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving env.txt to env.txt


In [ ]:
import os

if os.path.exists("env.txt"):
    os.rename("env.txt", "env")
    print("Renamed env.txt to env")
else:
    print("File not found. Make sure you uploaded env.txt.")

Renamed env.txt to env


In [ ]:
!pip install snowflake-connector-python python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
from dotenv import load_dotenv
load_dotenv("env")  # This loads the Snowflake credentials into the environment

True

In [ ]:
import snowflake.connector
import os

# Pull in credentials
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA")
)

# Run a simple test query
cur = conn.cursor()
cur.execute("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_DATABASE();")
for row in cur:
    print(row)

cur.close()
conn.close()

('SHIVNEETN', 'AST_REALESTATE_DB_RW', 'AST_REALESTATE_DB')


## Updating'InvestmentStyle' column with real-time data (from functional code)

In [ ]:
from snowflake.connector import connect
from snowflake.connector.pandas_tools import write_pandas
import os

# -----------------------------
# This script uploads the PortfolioGeneralInformation (PGI) DataFrame to Snowflake.
# It first ensures date columns are properly formatted,
# then loads the data into a staging table, and finally merges it into the target PGI table.
# -----------------------------

# Ensure date columns are in proper datetime.date format (required for Snowflake DATE fields)
pgi_df["OPENDATE"] = pd.to_datetime(pgi_df["OPENDATE"])
pgi_df["PERFORMANCEINCEPTIONDATE"] = pd.to_datetime(pgi_df["PERFORMANCEINCEPTIONDATE"])
pgi_df["TERMINATIONDATE"] = pd.to_datetime(pgi_df["TERMINATIONDATE"])

# Strip time component to make sure dates load into Snowflake DATE columns (not TIMESTAMP)
for col in ["OPENDATE", "PERFORMANCEINCEPTIONDATE", "TERMINATIONDATE"]:
    pgi_df[col] = pd.to_datetime(pgi_df[col]).dt.date

# -----------------------------
# Connect to Snowflake using environment credentials
# -----------------------------
conn = connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database="AST_REALESTATE_DB",
    schema="DBO"
)

# -----------------------------
# Create a staging table to temporarily hold PGI data
# -----------------------------
with conn.cursor() as cur:
    cur.execute("USE DATABASE AST_REALESTATE_DB")
    cur.execute("USE SCHEMA DBO")

    cur.execute("""
        CREATE OR REPLACE TEMP TABLE STG_PGI (
            PORTFOLIOCODE VARCHAR,
            NAME VARCHAR,
            INVESTMENTSTYLE VARCHAR,
            PORTFOLIOCATEGORY VARCHAR,
            OPENDATE DATE,
            PERFORMANCEINCEPTIONDATE DATE,
            BASECURRENCYCODE VARCHAR,
            BASECURRENCYNAME VARCHAR,
            ISBEGINOFDAYPERFORMANCE BOOLEAN,
            PRODUCTCODE VARCHAR,
            TERMINATIONDATE DATE
        )
    """)

# -----------------------------
# Upload DataFrame to Snowflake staging table
# -----------------------------
success, nchunks, nrows, _ = write_pandas(
    conn, df=pgi_df, table_name="STG_PGI", quote_identifiers=False
)
print(f"Uploaded {nrows} rows to staging.")

# -----------------------------
# Merge staging data into the main PortfolioGeneralInformation table
# Rows are matched by PORTFOLIOCODE
# Existing rows are updated; new rows are inserted
# -----------------------------
with conn.cursor() as cur:
    cur.execute("""
        MERGE INTO PortfolioGeneralInformation AS pgi
        USING STG_PGI AS stg
        ON pgi.PORTFOLIOCODE = stg.PORTFOLIOCODE

        WHEN MATCHED THEN UPDATE SET
            pgi.NAME = stg.NAME,
            pgi.INVESTMENTSTYLE = stg.INVESTMENTSTYLE,
            pgi.PORTFOLIOCATEGORY = stg.PORTFOLIOCATEGORY,
            pgi.OPENDATE = stg.OPENDATE,
            pgi.PERFORMANCEINCEPTIONDATE = stg.PERFORMANCEINCEPTIONDATE,
            pgi.BASECURRENCYCODE = stg.BASECURRENCYCODE,
            pgi.BASECURRENCYNAME = stg.BASECURRENCYNAME,
            pgi.ISBEGINOFDAYPERFORMANCE = stg.ISBEGINOFDAYPERFORMANCE,
            pgi.PRODUCTCODE = stg.PRODUCTCODE,
            pgi.TERMINATIONDATE = stg.TERMINATIONDATE

        WHEN NOT MATCHED THEN INSERT (
            PORTFOLIOCODE, NAME, INVESTMENTSTYLE, PORTFOLIOCATEGORY, OPENDATE,
            PERFORMANCEINCEPTIONDATE, BASECURRENCYCODE, BASECURRENCYNAME,
            ISBEGINOFDAYPERFORMANCE, PRODUCTCODE, TERMINATIONDATE
        ) VALUES (
            stg.PORTFOLIOCODE, stg.NAME, stg.INVESTMENTSTYLE, stg.PORTFOLIOCATEGORY,
            stg.OPENDATE, stg.PERFORMANCEINCEPTIONDATE, stg.BASECURRENCYCODE,
            stg.BASECURRENCYNAME, stg.ISBEGINOFDAYPERFORMANCE, stg.PRODUCTCODE,
            stg.TERMINATIONDATE
        )
    """)
    print("MERGE completed.")

Uploaded 5 rows to staging.
MERGE completed.


# **Data Flow & Architecture Documentation (WIP)**

## Source to Storage Mapping

| **Stage**                | **Description**                                                                                                                                 |
| ------------------------ | ----------------------------------------------------------------------------------------------------------------------------------------------- |
| **Source**               | Ticker-based REIT classification sources:                                                                                                       |
|                          |   • [REITNotes](https://www.reitnotes.com/reit-list/)                                                                                           |
|                          |   • [Nareit REIT Directory](https://www.reit.com/investing/reit-directory)                                                                      |
|                          |   • [SEC Master Ticker List](https://www.sec.gov/files/company_tickers_exchange.json) (CIK, exchange, ticker metadata)                          |
| **Extraction**           | `classify_style.py` loads and parses the above sources using `requests`, `BeautifulSoup`, and `pandas.read_html()`                              |
| **Reconciliation**       | Styles are resolved using hybrid logic in `resolve_style()`:                                                                                    |
|                          |   • If both REITNotes and Nareit agree → use that style                                                                                         |
|                          |   • If both disagree → assign `"Hybrid"`                                                                                                        |
|                          |   • If only one is present → assign its style                                                                                                   |
|                          |   • If neither found → assign `None`                                                                                                            |
| **Portfolio Generation** | `generate_pgi.py` creates 5 synthetic portfolios  → 2-3 portfolios per product across 2 products (REITProd001 to REITProd002)        |
| **Validation**           | `validate.py` asserts:                                                                                                                          |
|                          |   • All styles are in an approved whitelist                                                                                                     |
|                          |   • No more than 30% are missing (`None`)                                                                                                       |
|                          |   • Logs bad rows with `Ticker`, `InvestmentStyle`, and context to `style_classification.log`                                                   |
| **Enrichment**           | `generate_pgi_df()` attaches metadata: Portfolio name, open date, strategy, currency, etc.                                                      |
| **Storage**              | Final DataFrame (`pgi_df`) is uploaded to `STG_PGI` (a temp staging table in Snowflake) using `write_pandas()`                                  |
|                          | A `MERGE` SQL statement inserts new `PORTFOLIOCODE`s and updates existing rows in the `PortfolioGeneralInformation` table                       |
| **Consumption**          | Assette’s fact sheet engine queries `PortfolioGeneralInformation.INVESTMENTSTYLE` for classification, disclosure, filtering, and business logic |


## Data Flow Diagram

         +----------------------------+
         | SEC Master Ticker JSON     |
         |  ↳ https://sec.gov/...     |
         +----------------------------+
                     |
         +----------------------------+
         | REITNotes (read_html)      |
         |  ↳ https://reitnotes.com   |
         +----------------------------+
                     |
         +----------------------------+
         | Nareit Directory (scrape)  |
         |  ↳ https://reit.com/...    |
         +----------------------------+
                     |
                     v
         +-----------------------------+
         | classify_style.py           |   <-- classify_styles(ticker_list)
         +-----------------------------+
                     |
         +-----------------------------+
         | validate.py                 |   <-- assert_styles(df)
         |  ↳ logs to style_classification.log
         +-----------------------------+
                     |
         +-----------------------------+
         | generate_pgi.py             |   <-- generate_pgi_df(style_df)
         +-----------------------------+
                     |
         +-----------------------------+
         | main.py                     |   <-- orchestrates full pipeline
         |  ↳ write_pandas() to STG_PGI
         |  ↳ MERGE into PGI table
         +-----------------------------+
                     |
                     v
         +----------------------------------------------+
         | Snowflake Table: PortfolioGeneralInformation |
         |  ↳ INVESTMENTSTYLE + metadata inserted       |
         +----------------------------------------------+


## Transformation Logic Documentation

| **Transformation**              | **Description**                                                                                                   |
| ------------------------------- | ----------------------------------------------------------------------------------------------------------------- |
| **Web scraping**                | REITNotes and Nareit data extracted using BeautifulSoup, requests, and pandas                                     |
| **Hybrid Resolution Logic**     | If both sources agree → use it<br>If both disagree → `"Hybrid"`<br>If one available → use it<br>If neither → None |
| **Portfolio Allocation**        | 5 portfolios mapped under 2 products            |
| **Metadata Enrichment**         | Adds: `NAME`, `OPENDATE`, `STRATEGY`, `BASECURRENCY`, `PRODUCTCODE`, etc. using hardcoded/synthetic defaults      |
| **Validation (assert\_styles)** | Ensures styles conform to internal REIT/asset type whitelist                                                      |
| **Data Types**                  | Styles and metadata stored as `VARCHAR`, `BOOLEAN`, and `DATE` in Snowflake                                       |
| **Insert Logic**                | MERGE SQL performs insert-or-update behavior on `PortfolioGeneralInformation`                                     |
| **Logging**                     | All validation errors recorded in `style_classification.log` with timestamps and line-level diagnostics           |


## Technical Notes

- Modular pipeline split across:

  - classify_style.py → scraping + REIT classification

  - validate.py → validator for InvestmentStyle column

  - generate_pgi.py → 5-portfolio generation + enrichment

  - main.py → orchestrator

  - snowflake_loader.py → staging + merge to Snowflake

- All portfolio + product codes are synthetic (e.g. PORTFOLIOCODE = REITPORTFOLIO001 → REITProd001)

- Final output stored in Snowflake table: PortfolioGeneralInformation

- Merge SQL uses PORTFOLIOCODE as the primary key

- Designed to support both initial inserts and subsequent updates